In [25]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.dataset import *
from fastai.learner import *
from fastai.metrics import *
from fastai.core import *
from fastai.model import *
from fastai.lm_rnn import *
from collections import *
from fastai.conv_learner import *

In [2]:
normalize = torchvision.transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)

In [3]:
preprocess = torchvision.transforms.Compose([
   torchvision.transforms.Resize(256),
   torchvision.transforms.CenterCrop(240),
   torchvision.transforms.ToTensor(),
   normalize
])

In [4]:
class PasswordDataset(Dataset):
    def __init__(self, files, ys):
        self.files = files
        self.ys = ys
        
    def __getitem__(self, idx):
        return preprocess(PIL.Image.open(self.files[idx])).float().cuda(), self.ys[idx].long().cuda()
    
    def __len__(self): return len(self.ys)
    
    @classmethod
    def from_files(cls, img_path, y_path, cv_idxs):
        files = np.array(glob(img_path))

        with open(y_path) as f:
            ys = torch.Tensor([int(x) for x in f.read().split("\n") if x != ''])
        
        print(ys[cv_idxs])
        return cls(files[~cv_idxs], ys[~cv_idxs]), cls(files[cv_idxs], ys[cv_idxs])


In [5]:
image_paths = "/home/robert/Documents/typingviewer/data/*.jpg"

In [6]:
y_path = "/home/robert/Documents/typingviewer/data/y.csv"

In [7]:
dataset_size = len(glob(image_paths))

In [8]:
cv_idxs = get_cv_idxs(dataset_size, val_pct=0.5)

In [9]:
cv_idxs

array([1, 3])

In [10]:
trn_ds, val_ds = PasswordDataset.from_files(image_paths, y_path, cv_idxs)

tensor([ 7.,  9.])


In [11]:
trn_ds[1]

(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1008, -2.1008, -2.1008,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-1.9467, -1.9124, -1.9295,  ..., -1.9124, -1.8782, -1.9124],
          [-1.9980, -1.9980, -1.9638,  ..., -2.0837, -2.1008, -2.1008],
          [-2.0665, -2.0665, -2.0665,  ..., -1.9124, -1.9295, -1.9295]],
 
         [[-1.3179, -1.3004, -1.3004,  ...,  0.3978,  0.4153,  0.4153],
          [-1.3179, -1.3004, -1.3004,  ...,  0.4153,  0.4153,  0.4153],
          [-1.3004, -1.3004, -1.3004,  ...,  0.4328,  0.4328,  0.4503],
          ...,
          [-1.7556, -1.7206, -1.7381,  ..., -1.6681, -1.6155, -1.5980],
          [-1.8081, -1.8081, -1.7731,  ..., -1.6331, -1.6506, -1.6155],
          [-1.9132, -1.9132, -1.8782,  ..., -1.1078, -1.1078, -1.0903]],
 
         [[-0.8458, -0.8284, -0.8284,  ...,  0.7054,  0.7054,  0.6879],
          [-0.8458, -0.8284,

In [12]:
bs = 2

In [13]:
trn_dl = torch.utils.data.DataLoader(trn_ds, batch_size=bs)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=bs)

In [14]:
class ConvNet(nn.Module):
    def __init__(self, out_dim=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3))
        self.fc = nn.Linear(26*26*32, out_dim)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        #print(out.size())
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [15]:
myConv = SingleModel(ConvNet().cuda())

In [16]:
md = ModelData('model', trn_dl, val_dl)

In [17]:
def faux_loss(preds):
    print(preds)

In [43]:
myConv = SingleModel(ConvnetBuilder(resnet18, 200, False, False).model[:-1])

In [44]:
#myConv.model[:-1]

In [38]:
learner = Learner(md, myConv, opt_fn=optim.Adam, crit=F.nll_loss)

In [39]:
learner.fit(1*1e-3, 1)

epoch      trn_loss   val_loss       
    0      7.914818   4.674003  



[4.674002647399902]

In [20]:
class PasswordStatefulLSTM(nn.Module):
    def __init__(self, bs):
        super().__init__()
        
        self.cnn_final_width = 200
        self.number_rnn_layers = 1
        self.rnn_hidden_count = 100
        self.output_size = 10
        
        self.rnn = nn.LSTM(self.cnn_final_width, self.rnn_hidden_count, self.number_rnn_layers, dropout=0.5)
        self.l_out = nn.Linear(self.rnn_hidden_count, self.output_size)
        self.init_hidden(bs)
        self.conv = ConvNet(out_dim=self.cnn_final_width)
        
    def forward(self, cs):
        rnn_input = self.conv(cs).unsqueeze(1) #seq_len, batch, input_size
        #bs = cs[0].size(0)
        #if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(rnn_input, self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.output_size)
    
    def init_hidden(self, bs):
        #self.h = (V(torch.zeros(self.number_rnn_layers, bs, self.rnn_hidden_count)),
        #          V(torch.zeros(self.number_rnn_layers, bs, self.rnn_hidden_count)))
        self.h = (V(torch.zeros(1, 1, self.rnn_hidden_count)),
                  V(torch.zeros(1, 1, self.rnn_hidden_count)))

In [21]:
myLSTM = SingleModel(PasswordStatefulLSTM(bs).cuda())

/home/robert/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [22]:
learner = Learner(md, myLSTM, opt_fn=optim.Adam, crit=F.nll_loss)

In [23]:
learner.fit(1*1e-3, 100)

epoch      trn_loss   val_loss       
    0      2.248991   2.431012  
    1      2.014701   2.901056       
    2      1.812328   3.507516       
    3      1.640727   4.049956       
    4      1.498911   4.341283       
    5      1.388798   4.55842        
    6      1.304812   4.760144       
    7      1.238718   4.896334       
    8      1.185136   5.045734       
    9      1.140917   5.184613       
    10     1.10362    5.296159       
    11     1.071793   5.345508       
    12     1.044014   5.448779       
    13     1.019801   5.541713       
    14     0.998449   5.632837       
    15     0.979462   5.729972       
    16     0.962452   5.804691       
    17     0.947132   5.865543       
    18     0.933282   5.982412       
    19     0.920613   6.069868       
    20     0.909025   6.146122       
    21     0.898389   6.219546       
    22     0.888592   6.290355       
    23     0.879538   6.358792       
    24     0.871148   6.427435       
    25     0.8633

[8.299017906188965]